In [41]:
import chardet
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd

fenda = 0.1

voltagem = [4.0, 7.0, 9.0, 9.5, 10.0]
corrente = [0.45, 0.56, 0.68, 0.69, 0.70]
resistencia = 0.2

resistividade = []
temperatura = []

for i in range(np.size(voltagem)):
    resistividade.append(5.65*((voltagem[i]/corrente[i]) - resistencia)/0.93)
    temperatura.append(103 + 38.1*resistividade[i] - 0.095*(resistividade[i]*resistividade[i]) + 0.000248*(resistividade[i]*resistividade[i]*resistividade[i]))
    
print(resistividade)
print(temperatura)

for volt in voltagem:
    print(volt)
    with open(f'data/0.1mm{volt}v.txt', 'rb') as f:
            enc = chardet.detect(f.read())  # or readline if the file is large

        print(enc['encoding'])
        # file = pd.read_csv(f'data/0.1mm{volt}v.txt', header = 0, sep='\s+', encoding=enc['encoding'])
        
        # print(file)
        
        



IndentationError: unindent does not match any outer indentation level (<tokenize>, line 27)